<p align="center">
<img src="https://github.com/datacamp/r-live-training-template/blob/master/assets/datacamp.svg?raw=True" alt = "DataCamp icon" width="50%">
<br>
<h1 align="center">Cleaning Data in R Live Training</h1>
</p>
<br>


Welcome to this hands-on training where you'll identify issues in a dataset and clean it from start to finish using R. It's often said that data scientists spend 80% of their time cleaning and manipulating data and only about 20% of their time analyzing it, so cleaning data is an important skill to master!

In this session, you will:

- Examine a dataset and identify its problem areas, and what needs to be done to fix them.
-Convert between data types to make analysis easier.
- Correct inconsistencies in categorical data.
- Deal with missing data.
- Perform data validation to ensure every value makes sense.

## **The Dataset**

The dataset we'll use is a CSV file named `nyc_airbnb.csv`, which contains data on [*Airbnb*](https://www.airbnb.com/) listings in New York City. It contains the following columns:

- `listing_id`: The unique identifier for a listing
- `name`: The description used on the listing
- `host_id`: Unique identifier for a host
- `host_name`: Name of host
- `nbhood_full`: Name of borough and neighborhood
- `coordinates`: Coordinates of listing _(latitude, longitude)_
- `room_type`: Type of room 
- `price`: Price per night for listing
- `nb_reviews`: Number of reviews received 
- `last_review`: Date of last review
- `reviews_per_month`: Average number of reviews per month
- `availability_365`: Number of days available per year
- `avg_rating`: Average rating (from 0 to 5)
- `avg_stays_per_month`: Average number of stays per month
- `pct_5_stars`: Percent of reviews that were 5-stars
- `listing_added`: Date when listing was added


In [61]:
# Install packages
install.packages("readr")
install.packages("dplyr")
install.packages("stringr")
install.packages("ggplot2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [0]:
# Load packages
library(readr)
library(dplyr)
library(stringr)
library(ggplot2)

In [63]:
# Load dataset
airbnb <- read_csv("https://raw.githubusercontent.com/datacamp/cleaning-data-in-r-live-training/master/assets/nyc_airbnb.csv")

Parsed with column specification:
cols(
  listing_id = col_double(),
  description = col_character(),
  host_id = col_double(),
  host_name = col_character(),
  nbhood_full = col_character(),
  coordinates = col_character(),
  room_type = col_character(),
  price = col_character(),
  nb_reviews = col_double(),
  last_review = col_character(),
  reviews_per_month = col_double(),
  availability_365 = col_double(),
  avg_rating = col_double(),
  avg_stays_per_month = col_double(),
  pct_5_stars = col_double(),
  listing_added = col_character()
)



In [64]:
head(airbnb)

listing_id,description,host_id,host_name,nbhood_full,coordinates,room_type,price,nb_reviews,last_review,reviews_per_month,availability_365,avg_rating,avg_stays_per_month,pct_5_stars,listing_added
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
13740704,"Cozy,budget friendly, cable inc, private entrance!",20583125,Michel,"Brooklyn, Flatlands","(40.63222, -73.93398)",Private room,$45,10,12/12/2018,0.70,85,4.100954,12.0,0.6094315,06/08/2018
22005115,Two floor apartment near Central Park,82746113,Cecilia,"Manhattan, Upper West Side","(40.78761, -73.96862)",Entire home/apt,$135,1,06/30/2019,1.00,145,3.367600,1.2,0.7461346,12/25/2018
21667615,Beautiful 1BR in Brooklyn Heights,78251,Leslie,"Brooklyn, Brooklyn Heights","(40.7007, -73.99517)",Entire home/apt,$150,0,NA,NA,65,NA,NA,NA,08/15/2018
6425850,"Spacious, charming studio",32715865,Yelena,"Manhattan, Upper West Side","(40.79169, -73.97498)",Entire home/apt,$86,5,09/23/2017,0.13,0,4.763203,6.0,0.7699471,03/20/2017
22986519,Bedroom on the lively Lower East Side,154262349,Brooke,"Manhattan, Lower East Side","(40.71884, -73.98354)",Private room,$160,23,06/12/2019,2.29,102,3.822591,27.6,0.6493831,10/23/2020
271954,Beautiful brownstone apartment,1423798,Aj,"Manhattan, Greenwich Village","(40.73388, -73.99452)",Entire home/apt,$150,203,06/20/2019,2.22,300,4.478396,243.6,0.7434997,12/15/2018


## Diagnosing data cleaning problems

We'll need to get a good look at the data frame in order to identify any problems that may cause issues during an analysis. There are a variety of functions (both from base R and `dplyr`) that can help us with this:

1. `head()` to look at the first few rows of the data
2. `glimpse()` to get a summary of the variables' data types
3. `summary()` to compute summary statistics of each variable and display the number of missing values
4. `duplicated()` to find duplicates


In [65]:
# Print the first few rows of data
head(airbnb)

listing_id,description,host_id,host_name,nbhood_full,coordinates,room_type,price,nb_reviews,last_review,reviews_per_month,availability_365,avg_rating,avg_stays_per_month,pct_5_stars,listing_added
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
13740704,"Cozy,budget friendly, cable inc, private entrance!",20583125,Michel,"Brooklyn, Flatlands","(40.63222, -73.93398)",Private room,$45,10,12/12/2018,0.70,85,4.100954,12.0,0.6094315,06/08/2018
22005115,Two floor apartment near Central Park,82746113,Cecilia,"Manhattan, Upper West Side","(40.78761, -73.96862)",Entire home/apt,$135,1,06/30/2019,1.00,145,3.367600,1.2,0.7461346,12/25/2018
21667615,Beautiful 1BR in Brooklyn Heights,78251,Leslie,"Brooklyn, Brooklyn Heights","(40.7007, -73.99517)",Entire home/apt,$150,0,NA,NA,65,NA,NA,NA,08/15/2018
6425850,"Spacious, charming studio",32715865,Yelena,"Manhattan, Upper West Side","(40.79169, -73.97498)",Entire home/apt,$86,5,09/23/2017,0.13,0,4.763203,6.0,0.7699471,03/20/2017
22986519,Bedroom on the lively Lower East Side,154262349,Brooke,"Manhattan, Lower East Side","(40.71884, -73.98354)",Private room,$160,23,06/12/2019,2.29,102,3.822591,27.6,0.6493831,10/23/2020
271954,Beautiful brownstone apartment,1423798,Aj,"Manhattan, Greenwich Village","(40.73388, -73.99452)",Entire home/apt,$150,203,06/20/2019,2.22,300,4.478396,243.6,0.7434997,12/15/2018


**Problems so far:**
1. Multiple pieces of information in one value:
  - `coordinates` are easier to work with when separated into latitude and longitude
  - `nbhood_full` contains both the borough name (i.e. Manhattan, Brooklyn, etc.) and the neighborhood name (i.e. Lower East Side)
2. `price` has a $ attached

In [66]:
# Inspect data types
glimpse(airbnb)

Rows: 10,019
Columns: 16
$ listing_id          <dbl> 13740704, 22005115, 21667615, 6425850, 22986519, …
$ description         <chr> "Cozy,budget friendly, cable inc, private entranc…
$ host_id             <dbl> 20583125, 82746113, 78251, 32715865, 154262349, 1…
$ host_name           <chr> "Michel", "Cecilia", "Leslie", "Yelena", "Brooke"…
$ nbhood_full         <chr> "Brooklyn, Flatlands", "Manhattan, Upper West Sid…
$ coordinates         <chr> "(40.63222, -73.93398)", "(40.78761, -73.96862)",…
$ room_type           <chr> "Private room", "Entire home/apt", "Entire home/a…
$ price               <chr> "$45", "$135", "$150", "$86", "$160", "$150", "$2…
$ nb_reviews          <dbl> 10, 1, 0, 5, 23, 203, 0, 2, 5, 8, 5, 2, 21, 0, 0,…
$ last_review         <chr> "12/12/2018", "06/30/2019", NA, "09/23/2017", "06…
$ reviews_per_month   <dbl> 0.70, 1.00, NA, 0.13, 2.29, 2.22, NA, 0.08, 0.15,…
$ availability_365    <dbl> 85, 145, 65, 0, 102, 300, 219, 353, 365, 9, 0, 24…
$ avg_rating          <dbl>

3. Columns like `coordinates` and `price` are factors instead of numeric values.
4. Columns with dates like `last_review` and `listing_added` are factors instead of the `Date` data type.

In [0]:
# Examine summary statistics and missing values
summary(airbnb)

       X           listing_id                                  name     
 Min.   :    1   Min.   :    3831                                :   5  
 1st Qu.: 2506   1st Qu.: 9674772   Beautiful Brooklyn Brownstone:   5  
 Median : 5010   Median :20070296   New york Multi-unit building :   5  
 Mean   : 5010   Mean   :19276341   Hillside Hotel               :   4  
 3rd Qu.: 7514   3rd Qu.:29338637   Home away from home          :   4  
 Max.   :10019   Max.   :36487245   Brooklyn Apartment           :   3  
                                    (Other)                      :9993  
    host_id                 host_name                          nbhood_full  
 Min.   :     2787   Michael     :  89   Brooklyn, Bedford-Stuyvesant: 777  
 1st Qu.:  7910880   David       :  85   Brooklyn, Williamsburg      : 766  
 Median : 31651673   Sonder (NYC):  66   Manhattan, Harlem           : 541  
 Mean   : 67959227   Alex        :  52   Brooklyn, Bushwick          : 502  
 3rd Qu.:107434423   Daniel    

5. There are 2075 missing values in `reviews_per_month`, `avg_rating`, `nb_stays`, and `pct_5_stars`.
6. The max of `avg_rating` is above 5 (out of range value)
7. There are inconsistencies in the categories of `room_type`, i.e. `"Private"`, `"Private room"`, and `"PRIVATE ROOM"`.

In [0]:
# Find data with duplicated listing_id
airbnb %>%
  filter(duplicated(listing_id))

listing_id,description,host_id,host_name,nbhood_full,coordinates,room_type,price,nb_reviews,last_review,reviews_per_month,availability_365,avg_rating,nb_stays,pct_5_stars,listing_added
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>
7319856,450ft Square Studio in Gramercy NY,11773680,Adam,"Manhattan, Kips Bay","(40.73813, -73.98098)",Entire home/apt,$280,4,2016-05-22,0.09,225,3.903764,4.8,0.7563810,2015-11-17
17861841,THE CREATIVE COZY ROOM,47591528,Janessa,"Brooklyn, Sheepshead Bay","(40.59211, -73.94126999999997)",Private room,$99,13,2019-05-23,0.52,82,4.806590,15.6,0.9374216,2018-11-17
35646737,"Private Cabins @ Chelsea, Manhattan",117365574,Maria,"Manhattan, Chelsea","(40.74946, -73.99627)",Private room,$85,1,2019-06-22,1.00,261,4.951714,1.2,0.6713879,2018-12-17
15027024,Newly renovated 1bd on lively & historic St Marks,8344620,Ethan,"Manhattan, East Village","(40.72693, -73.98385)",Entire home/apt,$180,10,2018-12-31,0.30,0,3.869729,12.0,0.7725126,2018-06-27
4244242,Best Bedroom in Bedstuy/Bushwick. Ensuite bathroom,22023014,BrooklynSleeps,"Brooklyn, Bedford-Stuyvesant","(40.69496, -73.93949)",Private room,$73,110,2019-06-23,1.96,323,4.962314,132.0,0.8098820,2018-12-18
33831116,Sonder | Stock Exchange | Collected 1BR + Laundry,219517861,Sonder (NYC),"Manhattan, Financial District","(40.70621, -74.01199)",Entire home/apt,$229,5,2019-06-15,1.92,350,4.026379,6.0,0.6017374,2018-12-10
16518377,East Village 1BR Apt with all the amenities,3012457,Cody,"Manhattan, East Village","(40.7235, -73.97963)",Entire home/apt,$200,3,2018-07-10,0.16,0,4.676670,3.6,0.6944427,2018-01-04
22014840,Sunny Bedroom Only 1 Metro Stop to Manhattan,32093643,Scarlett,"Manhattan, Roosevelt Island","(40.76211, -73.94887)",Private room,$70,2,2018-01-07,0.11,0,4.024336,2.4,0.7194262,2017-07-04
33346762,2BR Apartment in Brownstone Brooklyn!,50321289,Avery,"Brooklyn, Bedford-Stuyvesant","(40.682, -73.95681)",Entire home/apt,$140,4,2019-06-14,1.58,4,4.013393,4.8,0.7195908,2018-12-09


8. Duplicates: there are 17 rows whose `listing_id` already appeared earlier in the dataset.

## What do we need to do?

**Data type issues**
1. Split `coordinates` into latitude and longitude and convert `numeric` data type.
2. Remove `$`s from `price` column and convert to `numeric`.
3. Convert `last_review` and `listing_added` to `Date`.

**Text & categorical data issues**
4. Split `nbhood_full` into separate neighborhood and borough columns.
5. Collapse the categories of `room_type` so that they're consistent.

**Data range issues**
6. Fix the `rating` column so it doesn't exceed `5`.

**Missing data issues**
7. Further investigate the missing data and decide how to handle them.

**Duplicate data issues**
8. Further investigate duplicate data points and decide how to handle them.

***But also...***
- We need to validate our data using various sanity checks

# **Q & A**

## Cleaning the data


### Data type issues

In [0]:
# What does the data look like?
head(airbnb)

#### **Task 1:** Split `coordinates` into latitude and longitude and convert `numeric` data type.


In [0]:
lat_lon <- airbnb$coordinates %>%
  # Remove left parentheses
  str_remove_all(fixed("(")) %>%
  # Remove right parentheses
  str_remove_all(fixed(")")) %>%
  # Split latitude and longitude
  str_split(", ", simplify = TRUE) %>%
  # Convert from matrix to data frame
  as.data.frame(stringsAsFactors = FALSE) %>%
  # Rename columns
  rename(latitude = V1, longitude = V2)

In [0]:
airbnb <- airbnb %>%
  # Combine lat_lon with original data frame
  cbind(lat_lon) %>%
  # Convert to numeric
  mutate(latitude = as.numeric(latitude),
        longitude = as.numeric(longitude)) %>%
  # Remove coordinates column
  select(-coordinates)

#### **Task 2:** Remove `$`s from `price` column and convert to `numeric`.

In [0]:
# Remove $, save as price
price_clean <- airbnb$price %>%
  str_remove_all(fixed("$"))

In [70]:
# Convert to numeric and add to data frame
airbnb <- airbnb %>%
  mutate(price = as.numeric(price_clean))

Warning message:
“NAs introduced by coercion”


#### **Task 3:** Convert `last_review` and `listing_added` to `Date`.

<img src="https://raw.githubusercontent.com/datacamp/cleaning-data-in-r-live-training/master/assets/date_formats.png" alt="%d = day number, %m = month number, %Y = 4 digit year, %y = 2 digit year, %B = month, %b = month abbreviation" style="width: 200px;"/>

In [79]:
airbnb %>%
  mutate(last_review = as.Date(last_review, format = "%m/%d/%Y"))

listing_id,description,host_id,host_name,nbhood_full,room_type,price,nb_reviews,last_review,reviews_per_month,availability_365,avg_rating,avg_stays_per_month,pct_5_stars,listing_added,latitude,longitude
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
13740704,"Cozy,budget friendly, cable inc, private entrance!",20583125,Michel,"Brooklyn, Flatlands",Private room,45,10,2018-12-12,0.70,85,4.100954,12.0,0.6094315,06/08/2018,40.63222,-73.93398
22005115,Two floor apartment near Central Park,82746113,Cecilia,"Manhattan, Upper West Side",Entire home/apt,135,1,2019-06-30,1.00,145,3.367600,1.2,0.7461346,12/25/2018,40.78761,-73.96862
21667615,Beautiful 1BR in Brooklyn Heights,78251,Leslie,"Brooklyn, Brooklyn Heights",Entire home/apt,150,0,NA,NA,65,NA,NA,NA,08/15/2018,40.70070,-73.99517
6425850,"Spacious, charming studio",32715865,Yelena,"Manhattan, Upper West Side",Entire home/apt,86,5,2017-09-23,0.13,0,4.763203,6.0,0.7699471,03/20/2017,40.79169,-73.97498
22986519,Bedroom on the lively Lower East Side,154262349,Brooke,"Manhattan, Lower East Side",Private room,160,23,2019-06-12,2.29,102,3.822591,27.6,0.6493831,10/23/2020,40.71884,-73.98354
271954,Beautiful brownstone apartment,1423798,Aj,"Manhattan, Greenwich Village",Entire home/apt,150,203,2019-06-20,2.22,300,4.478396,243.6,0.7434997,12/15/2018,40.73388,-73.99452
33473567,Hamilton Heights Full Floor Garden Apt,33388853,Christine,"Manhattan, Harlem",Entire home/apt,200,0,NA,NA,219,NA,NA,NA,05/05/2018,40.83090,-73.94308
14218742,Luxury/3bedroom/3bthrm/Privateprkng/beach/rstrnts,78824908,Ilona,"Brooklyn, Sheepshead Bay",Entire home/apt,224,2,2018-06-22,0.08,353,4.091228,2.4,0.6903933,12/17/2017,40.58531,-73.93811
15125599,Beautiful One Bedroom Apartment Near Central Park,3191545,Kyle,"Manhattan, Theater District",Entire home/apt,169,5,2017-06-25,0.15,365,3.562947,6.0,0.7045380,12/20/2016,40.76100,-73.98522


In [0]:
?strptime